In [1]:
import os, time, re, random, glob, json, jieba
import numpy as np
import pandas as pd
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    default_data_collator,
    TextGenerationPipeline
)

W0222 16:02:19.297271 140704479725184 redirects.py:28] NOTE: Redirects are currently not supported in Windows or MacOs.


In [2]:
# device="cuda:0"
device="cpu"

In [3]:
CLEAN_TEXT_PATTERN = re.compile(r"[\r\n]")

def clean_text(text):
    return CLEAN_TEXT_PATTERN.sub("", text)

# SFT

In [3]:
import sentencepiece
model_file = "/Users/zeyesun/Documents/Data/models/pangu-350M/vocab.model"
sp = sentencepiece.SentencePieceProcessor()
sp.Load(model_file=model_file)

True

In [4]:
for i in range(10):
    print(sp.id_to_piece(i))

<unk>
<s>
</s>
▃
▂
<sep>
<pad>
<mask>
<eod>
<eot>


In [4]:
# model_name_or_path = "D:\\Data\\models\\pangu_2_6B"
# model_name_or_path = "D:\\Data\\models\\pangu-350M"
model_name_or_path = "/Users/zeyesun/Documents/Data/models/pangu-350M"
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)
tokenizer.add_special_tokens({'eos_token': "<eot>", 'pad_token': "<pad>", "sep_token": "<sep>"})

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.
Using unk_token, but it is not set yet.


0

In [5]:
model = AutoModelForCausalLM.from_pretrained(model_name_or_path, trust_remote_code=True, use_cache=False)
model.resize_token_embeddings(len(tokenizer.sp))
model.config.end_token_id = tokenizer.eos_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.max_length_prompt = 200
model.to(device)
# print(model.device)

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Some weights of GPTPanguForCausalLM were not initialized from the model checkpoint at /Users/zeyesun/Documents/Data/models/pangu-350M and are newly initialized: ['transformer.h.9.attn.bias', 'transformer.h.0.attn.bias', 'transformer.h.20.attn.masked_bias', 'transformer.h.12.attn.bias', 'transformer.h.20.attn.bias', 'transformer.h.19.attn.bias', 'transformer.h.22.attn.bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.14.attn.masked_bias', 'transformer.h.15.attn.bias', 'transformer.h.21.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.21.attn.bias', 'transformer.h.7.attn.masked_bias', 'lm_head.weight', 'transform

GPTPanguForCausalLM(
  (transformer): GPTPanguModel(
    (wte): Embedding(40000, 1024)
    (wpe): Embedding(1024, 1024)
    (wqe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPTPanguBlock(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPTPanguAttention(
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (c_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTPanguMLP(
          (c_fc): Linear(in_features=1024, out_features=4096, bias=True)
          (c_proj): Linear(in_features=4096, out_features

### Data Processing

In [6]:
# f = "/Users/zeyesun/Documents/Data/raw/baike_qa2019/baike_qa_train.json"
f = "D:\\Data\\raw\\baike_qa_train.json"
items = []
lens_prompt = []
lens_label = []
with open(f, "r", encoding="utf-8") as r:
    while True:
        line = r.readline()
        if not line:
            break
        item = json.loads(line.strip("\n"))
        prompt = clean_text(item['title'] if len(item['title']) > len(item['desc']) else item['desc'])
        label = clean_text(item['answer'])
        items.append(item)
        lens_prompt.append(len(prompt))
        lens_label.append(len(label))
print(len(items))
print(np.percentile(lens_prompt, np.arange(90, 101)))
print(np.percentile(lens_label, np.arange(90, 101)))

1425170


In [43]:
i = 789
items[i]

{'qid': 'qid_7770627748113178417',
 'category': '生活-保健养生',
 'title': '减肥健身我要下载一套很好的有氧健身操，请问哪里有下载地址，谢谢！ ',
 'desc': '我要下载一套很好的有氧操，请问哪里有下载地址，谢谢！',
 'answer': '我看练太极拳就可以，该运动也是有氧健身的运动，长期锻炼还可以治疗很多慢性病。'}

In [44]:
items[i]['answer']

'我看练太极拳就可以，该运动也是有氧健身的运动，长期锻炼还可以治疗很多慢性病。'

### prediction

In [6]:
checkpoint = "/Users/zeyesun/Documents/Data/output/sft/pangu-350M/checkpoint-12000/pytorch_model.bin"
st = torch.load(checkpoint, map_location="cpu")
model.load_state_dict(st)

<All keys matched successfully>

In [7]:
text_generator = TextGenerationPipeline(model, tokenizer, device=device)

The model 'GPTPanguForCausalLM' is not supported for . Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'GPT2LMHeadModel', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegatronBertForCausalLM', 'MvpForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausalLM', 'RobertaForCausalLM', 'RoFormerForCausalLM', 'Speech2Text2ForCausalLM', 'TransfoXLLMHeadModel', 'TrOCRForCausalLM', 'XGLMForCausalLM', 'XLMWithLMHeadModel', 'XLMProphetNetForCausalLM', 'XLMRobertaForCausalLM', 'XLMRobertaX

In [18]:
f = "/Users/zeyesun/Documents/Data/raw/baike_qa2019/baike_qa_train.json"
i = 0
prompts = []
prompts_processed = []
labels = []
with open(f, "r", encoding="utf-8") as r:
    while True:
        line = r.readline()
        if not line:
            break
        item = json.loads(line.strip("\n"))
        prompt = clean_text(item['title'] if len(item['title']) > len(item['desc']) else item['desc'])
        label = clean_text(item['answer'])
        prompt_processed = prompt + tokenizer.sep_token + "模型回答:"
        prompts.append(prompt)
        prompts_processed.append(prompt_processed)
        labels.append(label)
        i += 1
        if i > 1000:
            break

In [24]:
i = 10
j = 20
t1 = time.time()
results = text_generator(prompts_processed[i:j], max_length=100)
print(f"Finished prediction, time taken: {time.time()-t1}")
for prompt, res, label in zip(prompts[i:j], results[i:j], labels[i:j]):
    model_answer = res[0]['generated_text'].split("模型回答:")[1].replace("<eot>", "")
    print(f"prompt: {prompt}\nmodel answer: {model_answer}\nlabel: {label}\n\n")

Input length of input_ids is 655, but `max_length` is set to 100. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


Finished prediction, time taken: 318.54285526275635


In [23]:
prompts_processed[i:j]

['有6区无尽之海众神之子工会的吗?你们会怎么了?看不见人了,解散了 <sep>模型回答:',
 '老师好，青青稞酒明天再跌就可以建仓了吗？谢谢。 <sep>模型回答:',
 'onthesouthofthecity是在城内还是成外？ <sep>模型回答:',
 '古代官职升迁又怎么说的，比如：拜。。。最好还有例句！<sep>模型回答:',
 '在倾斜的离心管中加入和密度小于水的小球，为什么小球会下降？请给出解题步骤。<sep>模型回答:',
 '邯郸市哪里有卖纳威沙发 <sep>模型回答:',
 '如图所示，，A、B是圆的直径的两端，小张在A点，小王在B点，同时出发相向而行，他们在点第一次相遇，C点离A点80米；在D点第二次相遇，D点离B点60米。求这个圆的周长。<sep>模型回答:',
 'Word文档,一页半文字,怎样调整使其真好布满两页? <sep>模型回答:',
 '武汉江夏分局接到群众举报电话,说在天桥下有两个大炸弹,江夏分局民警与排弹专家一起奔赴现场,在桥梁下发现一个红色布袋,专家与民警小心翼翼的拆开布袋,里面还有几层报纸包裹,民警一层层拆开,最后发现果然是两个大炸弹:4个2和一对王…… 警察表示要严肃追究报警人责任<sep>模型回答:',
 '考点知识清单一1．地图上的方向（1）一般情况：面对地图，上_____下_____,左西右东（2）有指向标的地图：指向标的箭头指向_____方（3）有经纬网的地图：经线指示_____，纬线指示_____2．方格状或圆弧形的经纬网图（1）辨别南、北纬纬度数值向北递增为北纬纬度数值向南递增为南纬（2）辨别东、西经经度数值向东递增为_____经经度数值向西递增为_____经3．以南北极为中心的经纬网图（1）辨别南北图上标出的地球自转方向呈逆时针，则经纬网中心为_____极图上标出的地球自转方向考点知识清单一1．地图上的方向（1）一般情况：面对地图，上_____下_____,左西右东（2）有指向标的地图：指向标的箭头指向_____方（3）有经纬网的地图：经线指示_____，纬线指示_____2．方格状或圆弧形的经纬网图（1）辨别南、北纬纬度数值向北递增为北纬纬度数值向南递增为南纬（2）辨别东、西经经度数值向东递增为_____经经度数值向西递增为_____经3．以南北极为中心的经纬网图（1）辨别南北图上标出的地球自转方向呈

In [8]:
prompt = "内有吊车，牛腿高度怎么定？根据什么来定？生产的工艺要求？还是要考虑到其他因素？"
prompt_processed = prompt + tokenizer.sep_token + "模型回答:"
res = text_generator(prompt_processed, max_length=50)
model_answer = res[0]['generated_text'].split("模型回答:")[1].replace("<eot>", "")
# print(res)
print(f"prompt: {prompt}\nmodel answer: {model_answer}")

Building prefix dict from the default dictionary ...
I0222 16:03:10.048795 140704479725184 __init__.py:113] Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/wf/12stcn_s6zq56j9h3fnkv5lm0000gn/T/jieba.cache
I0222 16:03:10.050362 140704479725184 __init__.py:133] Loading model from cache /var/folders/wf/12stcn_s6zq56j9h3fnkv5lm0000gn/T/jieba.cache
Loading model cost 0.572 seconds.
I0222 16:03:10.622334 140704479725184 __init__.py:165] Loading model cost 0.572 seconds.
Prefix dict has been built successfully.
I0222 16:03:10.623375 140704479725184 __init__.py:166] Prefix dict has been built successfully.


[{'generated_text': '内有吊车，牛腿高度怎么定？根据什么来定？生产的工艺要求？还是要考虑到其他因素？<sep>模型回答:牛腿高度是指牛腿的长度,一般是指牛腿'}]


# Reward Model

In [9]:
model_name_or_path = "D:\\Data\\models\\pangu-350M"
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_cache=False, trust_remote_code=True)
tokenizer.add_special_tokens({'unk_token': "<unk>",
                                  'bos_token': "<s>",
                                  'eos_token': "<eot>",
                                  'pad_token': "<pad>",
                                  "sep_token": "<sep>"})

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


0

In [10]:
max_length = 1024
text = "你好，你是谁"
# text = "<|startoftext|>" + text + "<|endoftext|>"
res = tokenizer(text, max_length=max_length, truncation="longest_first", 
          return_tensors="pt", add_special_tokens=False)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\SUNZEY~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.501 seconds.
Prefix dict has been built successfully.


In [19]:
res.keys()
# torch.cat((res['input_ids'], res['input_ids']), axis=1)

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])